In [1]:
import matplotlib.pyplot as plt
import os
import time
import sys

In [ ]:
!mo --saved_model_dir ./000000001/ --data_type FP32 --output_dir models/resnet_50/FP32 --input_shape [1,180,180,3]

In [ ]:
!mo --saved_model_dir ./000000001/ --data_type FP16 --output_dir models/resnet_50/FP16 --input_shape [1,180,180,3]

In [2]:
%%writefile ov_classify_job.sh

INPUT=$1
FP_MODEL=$2
DEVICE=$3
OUTPUT=$4

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

MODELPATH=./models/resnet_50/${FP_MODEL}/saved_model.xml

python3 ov-classify.py -i $INPUT -m $MODELPATH -d $DEVICE -o $OUTPUT

Overwriting ov_classify_job.sh


In [3]:
%%writefile tf_classify_job.sh

INPUT=$1
OUTPUT=$2
    

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR


python3 tf-classify.py -i $INPUT -o $OUTPUT

Overwriting tf_classify_job.sh


In [4]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

      1 idc001skl,compnode,openvino-lts,intel-core,i5-6500te,intel-hd-530,ram8gb
      1 idc002mx8,compnode,openvino-latest,intel-core,i5-6500te,intel-hd-530,ram8gb,myriadx-8-vpu
      1 idc004nc2,compnode,openvino-latest,intel-core,i5-6500te,intel-hd-530,ram8gb,myriadx-1-vpu
      1 idc006kbl,compnode,openvino-latest,intel-core,i5-7500t,intel-hd-630,ram8gb
      1 idc007xv5,compnode,openvino-latest,intel-xeon,e3-1268l-v5,intel-hd-p530,ram32gb
      1 idc008u2g,compnode,openvino-latest,intel-atom,e3950,intel-hd-505,ram4gb,myriadx-1-vpu
      1 idc014,compnode,openvino-latest,intel-core,i7-8665ue,intel-uhd-620,ram16gb,myriadx-2-vpu
      1 idc016ai7,compnode,openvino-latest,intel-core,i7-8665ue,intel-uhd-620,ram16gb
      1 idc025,compnode,openvino-latest,intel-xeon,gold6258r,no-gpu,ram96gb,myriadx-8-vpu
      3 idc034,compnode,openvino-latest,intel-core,i7-7700,intel-hd-630,ram16gb,kmb
      2 idc039,compnode,openvino-latest,intel-xeon,platinum-8180,no-gpu,ram196gb
      2 idc040,compn

In [ ]:
!rm -rf results
# main_dir = 'test_images/'
# import os
# all_classes_folders = []
# classes = []
# all_test_images =[]
# for all_classes in os.listdir(main_dir):
#     all_classes_folders.append(all_classes)
#     for classes in os.listdir(main_dir+all_classes):
#         all_test_images.append(main_dir+all_classes+'/'+classes)
# print(all_test_images)

## CPU

### OpenVINO FP32

In [ ]:
job_id_cpu_ov_fp32 = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "test_images/ FP32 CPU ./results/classify_cpu_ov_fp32/" -N classify_cpu_ov_fp32 -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_cpu_ov_fp32[0])

### Tensorflow FP32

In [5]:
job_id_cpu_tf_fp32 = !qsub tf_classify_job.sh -l nodes=1:idc073 -F "test_images/ ./results/classify_cpu_tf_fp32/" -N classify_cpu_tf_fp32 -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_cpu_tf_fp32[0])

48305.v-qsvr-sb1.devcloud-edge


## GPU

### OpenVINO FP32

In [ ]:
job_id_gpu_ov_fp32 = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./test_images/glacier/60.jpg FP32 GPU ./results/classify_gpu_ov_fp32/ " -N classify_gpu_ov_fp32 -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_gpu_ov_fp32[0])

### OpenVINO FP16

In [ ]:
job_id_gpu_ov_fp16 = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./test_images/glacier/60.jpg FP16 GPU ./results/classify_gpu_ov_fp16/ " -N classify_gpu_ov_fp16 -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_gpu_ov_fp16[0])

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 
You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If the status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

In [6]:
from qarpo.demoutils import *
liveQstat()

Output(layout=Layout(border='1px solid gray', height='200px', overflow_y='auto', width='100%'))

Button(description='Stop', style=ButtonStyle())

## CPU - OpenVINO FP32

In [ ]:
job_id_cpu_ov_fp32 = job_id_cpu_ov_fp32[0].split('.')[0]
job_id_cpu_ov_fp32 = 'o'+job_id_cpu_ov_fp32
count = 0
while not os.path.exists('./classify_cpu_ov_fp32.{}'.format(job_id_cpu_ov_fp32)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
cpu_ov = open("./classify_cpu_ov_fp32.{}".format(job_id_cpu_ov_fp32), "r")
import re
stats_cpu_ov = {}
for fps_ov in cpu_ov:
    if re.search('FPS', fps_ov):
        print(fps_ov)
        stats_cpu_ov['FPS_OV'] = str(fps_ov.split(" ")[1])
    if re.search('Inference time', fps_ov):
        print(fps_ov)
        stats_cpu_ov['Time_OV'] = str(fps_ov.split(" ")[2])
    if re.search('Prediction', fps_ov):
        print(fps_ov)

## CPU - Tensorflow FP32

In [ ]:
job_id_cpu_tf_fp32 = job_id_cpu_tf_fp32[0].split('.')[0]
job_id_cpu_tf_fp32 = 'o'+job_id_cpu_tf_fp32
count = 0
while not os.path.exists('./classify_cpu_tf_fp32.{}'.format(job_id_cpu_tf_fp32)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
cpu_tf = open("./classify_cpu_tf_fp32.{}".format(job_id_cpu_tf_fp32), "r")
import re
stats_cpu_tf = {}
for fps_tf in cpu_tf:
    if re.search('FPS', fps_tf):
        print(fps_tf)
        stats_cpu_tf['FPS_TF'] = str(fps_tf.split(" ")[1])
    if re.search('Inference time', fps_tf):
        print(fps_tf)
        stats_cpu_tf['Time_TF'] = str(fps_tf.split(" ")[2])
    if re.search('Prediction', fps_tf):
        print(fps_tf)

## GPU - OpenVINO FP32

In [ ]:
job_id_gpu_ov_fp32 = job_id_gpu_ov_fp32[0].split('.')[0]
job_id_gpu_ov_fp32 = 'o'+job_id_gpu_ov_fp32
count = 0
while not os.path.exists('./classify_gpu_ov_fp32.{}'.format(job_id_gpu_ov_fp32)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
gpu_ov_fp32 = open("./classify_gpu_ov_fp32.{}".format(job_id_gpu_ov_fp32), "r")
import re
stats_gpu_fp32 = {}
for fps_ov in gpu_ov_fp32:
    if re.search('FPS', fps_ov):
        print(fps_ov)
        stats_gpu_fp32['FPS_OV'] = str(fps_ov.split(" ")[1])
    if re.search('Inference time', fps_ov):
        print(fps_ov)
        stats_gpu_fp32['Time_OV'] = str(fps_ov.split(" ")[2])
    if re.search('Prediction', fps_ov):
        print(fps_ov)

## GPU - OpenVINO FP16

In [ ]:
job_id_gpu_ov_fp16 = job_id_gpu_ov_fp16[0].split('.')[0]
job_id_gpu_ov_fp16 = 'o'+job_id_gpu_ov_fp16
count = 0
while not os.path.exists('./classify_gpu_ov_fp16.{}'.format(job_id_gpu_ov_fp16)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
gpu_ov_fp16 = open("./classify_gpu_ov_fp16.{}".format(job_id_gpu_ov_fp16), "r")
import re
stats_gpu_fp16 = {}
for fps_ov in gpu_ov_fp16:
    if re.search('FPS', fps_ov):
        print(fps_ov)
        stats_gpu_fp16['FPS_OV'] = str(fps_ov.split(" ")[1])
    if re.search('Inference time', fps_ov):
        print(fps_ov)
        stats_gpu_fp16['Time_OV'] = str(fps_ov.split(" ")[2])
    if re.search('Prediction', fps_ov):
        print(fps_ov)